In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import os
import torch
from src.data import PancreasDataset
from src.data.dataset import RAW_DIR

from torch.utils.data import DataLoader
from src.models.standard_unet import UNet
from src.utils import Logger
from src.losses import MulticlassDiceLoss
from torch.optim import Adam
from src.training import Trainer

In [45]:
sample_dirs = [os.path.join(RAW_DIR, sd) for sd in os.listdir(RAW_DIR)]
ids = sample_dirs[:2]
print(ids)
# dataset = PancreasDataset(ids)

['C:\\Users\\Usuario\\TFG\\digipanca\\data\\raw\\rtum1', 'C:\\Users\\Usuario\\TFG\\digipanca\\data\\raw\\rtum10']


In [46]:
train_dataset, test_dataset, train_ids, test_ids = PancreasDataset.create_train_test_split(ids, 0.5)

Loaded train-test split from file: train_test_split.json.
📊 Loading dataset... 1 patients found.
📊 Dataset loaded with 91 slices.
📊 Loading dataset... 1 patients found.
📊 Dataset loaded with 56 slices.


In [47]:
train_dls = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)
test_dls = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4
)

In [48]:
logger = Logger(log_dir="test/logs", verbosity="INFO")

In [49]:
model = UNet(1,5,32)
loss_fn = MulticlassDiceLoss()
optimizer = Adam(model.parameters(),lr=1e-4, weight_decay=1e-5)

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [51]:
trainer = Trainer(
    model,
    optimizer,
    loss_fn,
    device
)

In [52]:
trainer.fit(train_dls, test_dls, 1)

Epoch 1 [Val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.57s/it]


([0.9964405993620554], [0.9940736442804337])